In [3]:
import dlc_bci as bci

import torch
import numpy as np

from models import *
from callbacks import keep_best_model, store_best_model

from types import SimpleNamespace 


from torch import optim
from torch import nn
from torch import Tensor
from torch.autograd import Variable

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load

In [4]:
one_khz=False

train = SimpleNamespace()
train.X, train.y = bci.load(root='./data_bci', one_khz=one_khz)
print(str(type(train.X)), train.X.size())
print(str(type(train.y)), train.y.size())

test = SimpleNamespace()
test.X, test.y = bci.load(root='./data_bci', train=False, one_khz=one_khz)
print(str(type(test.X)), test.X.size())
print(str(type(test.y)), test.y.size())

<class 'torch.Tensor'> torch.Size([316, 28, 50])
<class 'torch.Tensor'> torch.Size([316])
<class 'torch.Tensor'> torch.Size([100, 28, 50])
<class 'torch.Tensor'> torch.Size([100])


In [2]:
def cross_validate_grid_search(
    modelClass, 
    train, 
    epochs=100,
    n_splits=5,
    grid_search_on=[
        # first optimize structure
        ("nb_layers", list(range(1, 7))),
        ("nb_hidden", [np.asscalar(n) for n in np.arange(40, 201, 40)]),
        ("activation", [nn.ReLU, nn.Tanh, nn.ELU]),
        # second optimize optimizer
        ("optimizer", [optim.Adam, optim.Adadelta, optim.Adamax]),
        # then optimize regularization
        ("weight_decay", [0] + [np.asscalar(wd) for wd in np.logspace(-6, -2, 5, base=np.e)]),
        ("dropout", [np.asscalar(d) for d in np.linspace(0, 0.30, 4)]),
        # finally re-optimize number of layers (very important parameter)
        ("nb_layers", list(range(1, 7)))
    ]
):
    """ Apply a simplified grid search on the parameters in "grid_search_on" and optimize
    one at a time (to keep the complexity linear).    
    Finally save the best configuration.
    """
    
    X_tr, y_tr = modelClass.prepare_data(train)
    X_tr = X_tr.to(device)
    y_tr = y_tr.to(device)

    print("Optimizing in order:", list(map(lambda x: x[0], grid_search_on)))
    # here we store the best parameters as we find them (initialized with just the first parameter)
    bestParams = {}
    param_map = dict(grid_search_on)
    for param in param_map.keys():
        bestParams[param] = param_map[param][0]
    
    for param in grid_search_on:
        modelScores = {
            "tr_scores": [],
            "va_scores": []
        }
        param_name = param[0]
        param_values = param[1]
        print("\nCross validation on", param_name+":")
        for n in param_values: # scan through the parameter values
            print("------", param_name, "=", str(n), "------")
            bestParams[param_name] = n
            model = modelClass(**bestParams)
            model.to(device)
            result = model.cross_validate(X_tr, y_tr, verbose=False, epochs=epochs, n_splits=n_splits)

            modelScores["tr_scores"].append(result["train_score"])
            modelScores["va_scores"].append(result["test_score"])
            print("Validation scores:", result["test_score"])
        
        # save the obtained scores 
        model.save_data(modelScores, "cross_validation/scores/"+param_name, pickle_protocol=0)
        
        # compute the mean validation score for each param 
        modelScores["va_scores"] = list(map(lambda scores: np.asscalar(np.median(scores)), modelScores["va_scores"]))
        # get the best parameter
        bestParams[param_name] = param_values[np.argmax(modelScores["va_scores"])]
        print("Best", param_name + ":", str(bestParams[param_name]) + ". Score:", np.max(modelScores["va_scores"]))
    
    model.save_data(grid_search_on, "cross_validation/tried_params", pickle_protocol=0)
    model.save_data(bestParams, "cross_validation/best_params", pickle_protocol=0)
    
    model.save_data(grid_search_on, "cross_validation/tried_params_")
    model.save_data(bestParams, "cross_validation/best_params_")
    # [np.asscalar(wd) for wd in np.logspace(-6, -1, 4)]
    # best_n = n_hidden_values[np.argmax(modelScores["va_scores"])]
    # print('Best #hidden units:', best_n)
    # print('Test score:',
    #       CNN2D(n)
    #       .fit(X_tr, y_tr, epochs=50)
    #       .score(X_te, y_te))

- cross validate gird search on `CNN2D_MaxPool`

In [6]:
cross_validate_grid_search(CNN2D_MaxPool, train, epochs=100)

Optimizing in order: ['nb_layers', 'nb_hidden', 'activation', 'optimizer', 'weight_decay', 'dropout', 'nb_layers']

Cross validation on nb_layers:
------ nb_layers = 1 ------


/cvlabdata1/home/sacchi/others/predict-finger-movement/modelWrapper.py:126: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  sum_loss_train += loss.data[0].item()


Validation scores: [0.75, 0.6507936507936508, 0.7619047619047619, 0.7142857142857143, 0.746031746031746]
------ nb_layers = 2 ------
Validation scores: [0.75, 0.7619047619047619, 0.7301587301587301, 0.7301587301587301, 0.746031746031746]
------ nb_layers = 3 ------
Validation scores: [0.75, 0.746031746031746, 0.7619047619047619, 0.7619047619047619, 0.746031746031746]
------ nb_layers = 4 ------
Validation scores: [0.78125, 0.7142857142857143, 0.7301587301587301, 0.7936507936507936, 0.7301587301587301]
------ nb_layers = 5 ------
Validation scores: [0.640625, 0.746031746031746, 0.7936507936507936, 0.7936507936507936, 0.8412698412698413]
------ nb_layers = 6 ------
Validation scores: [0.765625, 0.7936507936507936, 0.8095238095238095, 0.746031746031746, 0.7777777777777778]
Best nb_layers: 5. Score: 0.793650793651

Cross validation on nb_hidden:
------ nb_hidden = 40 ------
Validation scores: [0.71875, 0.7619047619047619, 0.8571428571428571, 0.8095238095238095, 0.7142857142857143]
------ n

- cross validate gird search on `CNN_1D_MaxPool`

In [7]:
cross_validate_grid_search(CNN_1D_MaxPool, train, epochs=100)

Optimizing in order: ['nb_layers', 'nb_hidden', 'activation', 'optimizer', 'weight_decay', 'dropout', 'nb_layers']

Cross validation on nb_layers:
------ nb_layers = 1 ------


/cvlabdata1/home/sacchi/others/predict-finger-movement/modelWrapper.py:126: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  sum_loss_train += loss.data[0].item()


Validation scores: [0.765625, 0.7777777777777778, 0.7936507936507936, 0.6825396825396826, 0.6349206349206349]
------ nb_layers = 2 ------
Validation scores: [0.734375, 0.7142857142857143, 0.8412698412698413, 0.8412698412698413, 0.6190476190476191]
------ nb_layers = 3 ------
Validation scores: [0.671875, 0.5396825396825397, 0.8412698412698413, 0.746031746031746, 0.5714285714285714]
------ nb_layers = 4 ------
Validation scores: [0.671875, 0.7936507936507936, 0.7936507936507936, 0.7619047619047619, 0.6666666666666666]
------ nb_layers = 5 ------
Validation scores: [0.578125, 0.746031746031746, 0.8412698412698413, 0.7142857142857143, 0.6984126984126984]
------ nb_layers = 6 ------
Validation scores: [0.625, 0.6984126984126984, 0.746031746031746, 0.5079365079365079, 0.7142857142857143]
Best nb_layers: 1. Score: 0.765625

Cross validation on nb_hidden:
------ nb_hidden = 40 ------
Validation scores: [0.796875, 0.8253968253968254, 0.8253968253968254, 0.7142857142857143, 0.6984126984126984]


- cross validate gird search on `CNN_1D_BatchNorm`

In [8]:
cross_validate_grid_search(CNN_1D_BatchNorm, train, epochs=100)

Optimizing in order: ['nb_layers', 'nb_hidden', 'activation', 'optimizer', 'weight_decay', 'dropout', 'nb_layers']

Cross validation on nb_layers:
------ nb_layers = 1 ------


/cvlabdata1/home/sacchi/others/predict-finger-movement/modelWrapper.py:126: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  sum_loss_train += loss.data[0].item()


Validation scores: [0.75, 0.7619047619047619, 0.6984126984126984, 0.7619047619047619, 0.7619047619047619]
------ nb_layers = 2 ------
Validation scores: [0.71875, 0.6825396825396826, 0.7777777777777778, 0.7301587301587301, 0.6031746031746031]
------ nb_layers = 3 ------
Validation scores: [0.546875, 0.5555555555555556, 0.6984126984126984, 0.7777777777777778, 0.5079365079365079]
------ nb_layers = 4 ------
Validation scores: [0.65625, 0.6666666666666666, 0.6825396825396826, 0.6507936507936508, 0.6666666666666666]
------ nb_layers = 5 ------
Validation scores: [0.5625, 0.6507936507936508, 0.6984126984126984, 0.6825396825396826, 0.5079365079365079]
------ nb_layers = 6 ------
Validation scores: [0.625, 0.5555555555555556, 0.47619047619047616, 0.5079365079365079, 0.5238095238095238]
Best nb_layers: 1. Score: 0.761904761905

Cross validation on nb_hidden:
------ nb_hidden = 40 ------
Validation scores: [0.734375, 0.6984126984126984, 0.8412698412698413, 0.6825396825396826, 0.6666666666666666

- cross validate gird search on `CNN_1D_BatchNorm_Dial`

In [9]:
cross_validate_grid_search(CNN_1D_BatchNorm_Dial, train, epochs=100)

Optimizing in order: ['nb_layers', 'nb_hidden', 'activation', 'optimizer', 'weight_decay', 'dropout', 'nb_layers']

Cross validation on nb_layers:
------ nb_layers = 1 ------


/cvlabdata1/home/sacchi/others/predict-finger-movement/modelWrapper.py:126: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  sum_loss_train += loss.data[0].item()


Validation scores: [0.703125, 0.6825396825396826, 0.6666666666666666, 0.5238095238095238, 0.6031746031746031]
------ nb_layers = 2 ------
Validation scores: [0.609375, 0.5873015873015873, 0.6666666666666666, 0.6031746031746031, 0.5238095238095238]
------ nb_layers = 3 ------
Validation scores: [0.578125, 0.6507936507936508, 0.6825396825396826, 0.7301587301587301, 0.6190476190476191]
------ nb_layers = 4 ------
Validation scores: [0.5, 0.6666666666666666, 0.746031746031746, 0.4126984126984127, 0.4444444444444444]
------ nb_layers = 5 ------
Validation scores: [0.546875, 0.6190476190476191, 0.47619047619047616, 0.5555555555555556, 0.5396825396825397]
------ nb_layers = 6 ------
Validation scores: [0.4375, 0.5396825396825397, 0.5238095238095238, 0.5079365079365079, 0.5238095238095238]
Best nb_layers: 1. Score: 0.666666666667

Cross validation on nb_hidden:
------ nb_hidden = 40 ------
Validation scores: [0.75, 0.7777777777777778, 0.6825396825396826, 0.7142857142857143, 0.5714285714285714]

- cross validate gird search on `CNN_1D_Residual`

In [10]:
cross_validate_grid_search(CNN_1D_Residual, train, epochs=100)

Optimizing in order: ['nb_layers', 'nb_hidden', 'activation', 'optimizer', 'weight_decay', 'dropout', 'nb_layers']

Cross validation on nb_layers:
------ nb_layers = 1 ------


/cvlabdata1/home/sacchi/others/predict-finger-movement/modelWrapper.py:126: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  sum_loss_train += loss.data[0].item()


Validation scores: [0.84375, 0.7142857142857143, 0.8571428571428571, 0.7301587301587301, 0.6507936507936508]
------ nb_layers = 2 ------
Validation scores: [0.78125, 0.746031746031746, 0.6984126984126984, 0.746031746031746, 0.6507936507936508]
------ nb_layers = 3 ------
Validation scores: [0.78125, 0.6984126984126984, 0.7777777777777778, 0.6984126984126984, 0.6666666666666666]
------ nb_layers = 4 ------
Validation scores: [0.796875, 0.746031746031746, 0.7619047619047619, 0.6666666666666666, 0.6507936507936508]
------ nb_layers = 5 ------
Validation scores: [0.78125, 0.746031746031746, 0.7619047619047619, 0.746031746031746, 0.6190476190476191]
------ nb_layers = 6 ------
Validation scores: [0.734375, 0.6666666666666666, 0.8412698412698413, 0.6984126984126984, 0.6984126984126984]
Best nb_layers: 2. Score: 0.746031746032

Cross validation on nb_hidden:
------ nb_hidden = 40 ------
Validation scores: [0.71875, 0.7619047619047619, 0.7619047619047619, 0.6666666666666666, 0.5238095238095238